<a href="https://colab.research.google.com/github/trngbro/AI/blob/main/Progress20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

init

In [1]:
!pip install python-sat==0.1.7.dev12

from pysat.solvers import Glucose3
import itertools


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.2 MB/s eta 0:00:00


In [45]:
from pysat.solvers import Glucose3
from itertools import combinations, product

def get_propositions(matrix):
    props = []
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            if matrix[i][j] != '' and matrix[i][j]!=0:
                props.append((i, j))
    return props

def get_neighbors(prop, matrix):
    neighbors = []
    i, j = prop
    for x in range(max(0, i-1), min(i+2, len(matrix))):
        for y in range(max(0, j-1), min(j+2, len(matrix[0]))):
            if matrix[x][y] != '' and matrix[x][y]!=0:
                neighbors.append((x, y))
    return neighbors

# Tạo ra CNF :>> nhưng mà lạ lắm
def generate_clauses(props, matrix):
    clauses = []
    for p in props:
        neighbors = get_neighbors(p, matrix)
        num_green = matrix[p[0]][p[1]]
        for comb in combinations(neighbors, num_green):
            clause = []
            for n in neighbors:
                if n in comb:
                    clause.append(get_index(n, props) + 1)
                else:
                    clause.append(-(get_index(n, props) + 1))
            clauses.append(clause)
        if num_green == 0:
            for n in neighbors:
                clauses.append([-1 * (get_index(n, props) + 1)])
    return clauses


def get_index(prop, props):
    return props.index(prop)

def solve_puzzle(matrix):
    props = get_propositions(matrix)
    clauses = generate_clauses(props, matrix)
    with Glucose3() as solver:
        for clause in clauses:
            solver.add_clause(clause)
        res = solver.solve()
        if res == False:
            return None
        result = [['red' for j in range(len(matrix[0]))] for i in range(len(matrix))]
        for p in props:
            if solver.get_model()[get_index(p, props)] > 0:
                result[p[0]][p[1]] = 'green'
        return result

In [47]:
# Test the function with example input
matrix = [
    [1, '', 1],
    ['', 4, ''],
    [1,'', 1],
]



print(solve_puzzle(matrix))

print(get_neighbors((1,1),matrix))

[['red', 'red', 'red'], ['red', 'red', 'red'], ['red', 'red', 'red']]
[(0, 0), (0, 2), (1, 1), (2, 0), (2, 2)]


In [ ]:
_matrix = [
	['',  2,  3, '', '',  0, '', '', '', ''],
	['', '', '', '',  3, '',  2, '', '',  6],
	['', '',  5, '',  5,  3, '',  5,  7,  4],
	['',  4, '',  5, '',  5, '',  6, '',  3],
	['', '',  4, '',  5, '',  6, '', '',  3],
	['', '', '',  2, '',  5, '', '', '', ''],
	[ 4, '',  1, '', '', '',  1,  1, '', ''],
	[ 4, '',  1, '', '', '',  1, '',  4, ''],
	['', '', '', '',  6, '', '', '', '',  4],
	['',  4,  4, '', '', '', '',  4, '', '']
] 

'''
[
	[R, G, G, R, R, R, R, R, G, G],
	[R, R, R, G, R, R, R, G, G, G],
	[R, R, G, G, G, R, R, G, G, G],
	[R, G, G, R, G, G, R, G, R, R],
	[R, G, R, R, R, G, G, G, G, R],
	[G, G, R, R, G, G, R, R, G, G],
	[G, R, R, R, G, R, R, R, R, G],
	[G, R, R, R, G, R, R, R, R, G],
	[G, G, R, R, G, G, R, R, G, G],
	[R, G, G, G, G, G, G, G, G, R]
] 
'''